# General IO

## Array Parsing

Throughout the entire package, array of numbers are very frequently used within the code as well as the inputs parameters to the methods. Thus, it is of great benefit to have a generic array parser to convert array into desired format, as well as enforcing array format for certain vairables.

In [1]:
from principia_materia import Fraction
from principia_materia.io_interface import parse_array

* We updated the methods of the built-in Fraction class, due to incompatibilities with numpy in python3.
* The `parse_array` method provides an easy way to creating an array, as demonstrated below. Supported delimiters for string format include space, comma, semicolon and newline, with hierarchy from low to high.

In [2]:
parse_array("""\
    r3/2 0.5 0.5
    -1/2 0.0 0.5
     3/4 0.5 0.0""", dtype=float)

[[0.8660254037844386, 0.5, 0.5], [-0.5, 0.0, 0.5], [0.75, 0.5, 0.0]]

In [3]:
parse_array("""\
     3/2 0.5 0.5
    -1/2 0.0 0.5
     3/4 0.5 0.0""", dtype=Fraction)

[[Fraction(3, 2), Fraction(1, 2), Fraction(1, 2)],
 [Fraction(-1, 2), Fraction(0, 1), Fraction(1, 2)],
 [Fraction(3, 4), Fraction(1, 2), Fraction(0, 1)]]

In [4]:
parse_array("""\
     3/2
    -1/2 0.0
     3/4 0.5 0.0""", dtype=Fraction, pad_to_shape=(3, 3), fillvalue=1)

array([[Fraction(3, 2), Fraction(1, 1), Fraction(1, 1)],
       [Fraction(-1, 2), Fraction(0, 1), Fraction(1, 1)],
       [Fraction(3, 4), Fraction(1, 2), Fraction(0, 1)]], dtype=object)

In [5]:
from principia_materia.translation_group import (
    get_structure,
    save_structure,
    Cluster,
    Crystal,
    CrystalFTG,
)

## Structure file

For all translation group related classes, including Lattice, Cluster, Crystal, CrystalFTG, LatticeFTG and Kpoints, we present a method ``get_structure`` to construct an instance from a YAML format file.
The file needs to contain all the required keywords present for the desired class and the values need to be in supported format. 

In case of arrays, the method can take either a YAML supported nested list or in a string of numbers with space and newline of comma and semicolon delimiters.

* ``vec``: lattice vectors, 2D array of float.
       
* ``atoms``: a list of single item dictionaries with the atom name as key and atom positions as the value.

* ``orbitals``: orbitals used in the structure, string, or a list of strings.

* ``supa``: supercell matrix, 2D array of int.

* ``pg``: name of the point group, string

This file format is easy to write by hand, thus can serve as an easy way to document a strcuture in study.
It is also used across the package to store structural information when needed. It can be easily converted into structure input files for different DFT engines on demand.

For example, if we have the follwing YAML string for a CrystalFTG class, we can pass it into the ``get_structure``  method, specifying the structure type with ``stype=CrystalFTG``, we will get a instance of the CrystalFTG class of rocksalt.

In [6]:
structure_crystalftg = """\
vec: |
    0.0 0.5 0.5
    0.5 0.0 0.5
    0.5 0.5 0.0
atoms:
    - Na:
        0.0 0.0 0.0
    - Cl:
        0.5 0.5 0.5
orbitals: p
supa: |
    1 0 0
    0 1 0
    0 0 1
"""

In [7]:
struct = get_structure(structure_crystalftg, stype=CrystalFTG)
struct.to_dict()

OrderedDict([('vec',
              array([[0. , 0.5, 0.5],
                     [0.5, 0. , 0.5],
                     [0.5, 0.5, 0. ]])),
             ('atoms',
              OrderedDict([('Na', array([[0., 0., 0.]])),
                           ('Cl', array([[0.5, 0.5, 0.5]]))])),
             ('orbitals', 'p'),
             ('supa',
              array([[1, 0, 0],
                     [0, 1, 0],
                     [0, 0, 1]]))])

The same can be achieved by passing in the path to a YAML file, here we have this **structure_crystalftg.yml** file:

In [8]:
cat structure_crystalftg.yml

vec: |
    0.0,0.5,0.5
    0.5,0.0,0.5
    0.5,0.5,0.0
atoms:
    - Na:
        0.0 0.0 0.0
    - Cl:
        0.5 0.5 0.5
orbitals: p
supa: |
    1 0 0
    0 1 0
    0 0 1



We can construct an instance with the same information:

In [9]:
struct = get_structure("structure_crystalftg.yml", stype=CrystalFTG)
struct.to_dict()

OrderedDict([('vec',
              array([[0. , 0.5, 0.5],
                     [0.5, 0. , 0.5],
                     [0.5, 0.5, 0. ]])),
             ('atoms',
              OrderedDict([('Na', array([[0., 0., 0.]])),
                           ('Cl', array([[0.5, 0.5, 0.5]]))])),
             ('orbitals', 'p'),
             ('supa',
              array([[1, 0, 0],
                     [0, 1, 0],
                     [0, 0, 1]]))])

In [10]:
struct = get_structure("""\
vec: |
    r3/2  1/2  0
    r3/2 -1/2  0
    0          0   15
atoms:
    - C: |
        0.0 0.0 0.0
        2/3 2/3 0
orbitals: p
supa: |
    1 0 0
    0 1 0
    0 0 1
""", stype=CrystalFTG)
struct.to_dict()

OrderedDict([('vec',
              array([[ 0.8660254,  0.5      ,  0.       ],
                     [ 0.8660254, -0.5      ,  0.       ],
                     [ 0.       ,  0.       , 15.       ]])),
             ('atoms',
              OrderedDict([('C',
                            array([[0.        , 0.        , 0.        ],
                                   [0.66666667, 0.66666667, 0.        ]]))])),
             ('orbitals', 'p'),
             ('supa',
              array([[1, 0, 0],
                     [0, 1, 0],
                     [0, 0, 1]]))])

A structure instance can also be easily saved into the file format:

In [11]:
print(save_structure(struct))

# CrystalFTG
vec:
  [[ 0.86602540,  0.50000000,  0.00000000],
   [ 0.86602540, -0.50000000,  0.00000000],
   [ 0.00000000,  0.00000000,  15.00000000]]
atoms:
  - C: 
      [[ 0.00000000,  0.00000000,  0.00000000],
       [ 0.66666667,  0.66666667,  0.00000000]]
orbitals: p
supa:
  [[ 1,  0,  0],
   [ 0,  1,  0],
   [ 0,  0,  1]]


## Format arrays

Since we are dealing with a lot of arrays, sometime we want to print them out to save into a file, or even just for inspection.
The `ArrayFormatter` class enables us to format 2D arrays into various ways.

`ArrayFormatter` is a class that behaves like a function. Once instantiated, the object can be called just like a function.
This allows us to reused all of the formatting code and achieve various styles of formats in minimum amount of code.

We can instantiate the class with configurations to format arrays in desired style, then use it to format the arrays we have.
Meanwhile, many frequently used styles are pre-configured and ready to use.

In [12]:
from principia_materia.io_interface import (
    ArrayFormatter, 
    format_array_bracket, 
    format_array_table, 
    tex_array_formatter,
)

See class docstring for usage details:

In [13]:
help(ArrayFormatter)

Help on class ArrayFormatter in module principia_materia.io_interface.array_io:

class ArrayFormatter(builtins.object)
 |  ArrayFormatter(prec=8, fmt=None, align='l', cell_length=1, delimiter=' ', pad=0, newline='\n', header='', footer='', line_begin='', line_end='', indent=0)
 |  
 |  Format 2D array.
 |  
 |  Object oriented class for formatting arrays.
 |  The mechanism is to setup the desired style of the formatter, with which
 |  one can format arrays, just like any formatter function.
 |  Some default styles will be provided below, for convenience.
 |  
 |  This class implements all the basic features of a formatter, so that any
 |  new styles can be achieved with minimum work by simply providing style
 |  specifications.
 |  
 |  Parameters
 |  ----------
 |  prec        : int, optional, default to 8
 |      Decimal points to keep for floating points.
 |      Can be directly set in `fmt`.
 |  fmt         : str, optional, default to None
 |      The string format for each array e

In [14]:
array = np.array([
    [0.0, 0.5, 0.5],
    [0.5, 0.0, 0.5],
    [0.5, 0.5, 0.0],
])
fraction_array = np.array([
    [Fraction(0, 1), Fraction(1, 2), Fraction(1, 2)],
    [Fraction(1, 2), Fraction(0, 1), Fraction(1, 2)],
    [Fraction(1, 2), Fraction(1, 2), Fraction(0, 1)],
])

In [15]:
print(format_array_bracket(array))
print(format_array_bracket(fraction_array))

[[ 0.00000000,  0.50000000,  0.50000000],
 [ 0.50000000,  0.00000000,  0.50000000],
 [ 0.50000000,  0.50000000,  0.00000000]]
[[0, 1/2, 1/2],
 [1/2, 0, 1/2],
 [1/2, 1/2, 0]]


In [16]:
print(format_array_table(array))
print(format_array_table(fraction_array))

 0.00000000  0.50000000  0.50000000
 0.50000000  0.00000000  0.50000000
 0.50000000  0.50000000  0.00000000
0 1/2 1/2
1/2 0 1/2
1/2 1/2 0


Here the `tex_array_formatter` function can return a formatter instance that uses the specified tex environment (default to "bmatrix").

In [17]:
tex_formatter = tex_array_formatter(texenv="bmatrix")
print(tex_formatter(array))

\begin{bmatrix}
   0.00000000 &&  0.50000000 &&  0.50000000 \\
   0.50000000 &&  0.00000000 &&  0.50000000 \\
   0.50000000 &&  0.50000000 &&  0.00000000 \\
\end{bmatrix}
